In [ ]:
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
cd ../

/


In [ ]:
!git clone https://github.com/facebookresearch/fastText.git

Cloning into 'fastText'...
remote: Enumerating objects: 3854, done.
remote: Total 3854 (delta 0), reused 0 (delta 0), pack-reused 3854
Receiving objects: 100% (3854/3854), 8.23 MiB | 15.18 MiB/s, done.
Resolving deltas: 100% (2416/2416), done.


In [ ]:
cd fastText

/fastText


In [ ]:
!sudo pip install .

Processing /fastText
  Created wheel for fasttext: filename=fasttext-0.9.2-cp36-cp36m-linux_x86_64.whl size=3084869 sha256=e92db880f0eb5e887e54f00d842bcdc6d1c2c76682a79cb48a519055c8c3f445
  Stored in directory: /tmp/pip-ephem-wheel-cache-up538kai/wheels/dd/ab/3c/3d74e0601246cef39c2174e7995f715a8e871e0173c6d99bf8
Successfully built fasttext


In [ ]:
import fasttext

In [ ]:
import numpy as np


# predefined class to import fastvector
class FastVector:
    def __init__(self, vector_file='', transform=None):
        self.word2id = {}
        self.id2word = []

        print('reading word vectors from %s' % vector_file)
        with open(vector_file, 'r') as f:
            (self.n_words, self.n_dim) = \
                (int(x) for x in f.readline().rstrip('\n').split(' '))
            self.embed = np.zeros((self.n_words, self.n_dim))
            for i, line in enumerate(f):
                elems = line.rstrip('\n').split(' ')
                self.word2id[elems[0]] = i
                self.embed[i] = elems[1:self.n_dim+1]
                self.id2word.append(elems[0])

        if transform is not None:
            print('Applying transformation to embedding')
            self.apply_transform(transform)

    def apply_transform(self, transform):
        transmat = np.loadtxt(transform) if isinstance(transform, str) else transform
        self.embed = np.matmul(self.embed, transmat)

    def export(self, outpath):

        fout = open(outpath, "w")

        fout.write(str(self.n_words) + " " + str(self.n_dim) + "\n")
        for token in self.id2word:
            vector_components = ["%.6f" % number for number in self[token]]
            vector_as_string = " ".join(vector_components)

            out_line = token + " " + vector_as_string + "\n"
            fout.write(out_line)

        fout.close()


    @classmethod
    def cosine_similarity(cls, vec_a, vec_b):
        """Compute cosine similarity between vec_a and vec_b"""
        return np.dot(vec_a, vec_b) / \
            (np.linalg.norm(vec_a) * np.linalg.norm(vec_b))

    def __contains__(self, key):
        return key in self.word2id

    def __getitem__(self, key):
        return self.embed[self.word2id[key]]

In [ ]:
from fasttext import load_model

ur_dictionary = FastVector(vector_file='/content/gdrive/MyDrive/Data_for_wordEmbeddings/roman.vec')
eng_dictionary = FastVector(vector_file='/content/gdrive/MyDrive/Data_for_wordEmbeddings/try_wiki.simple.vec')

reading word vectors from /content/gdrive/MyDrive/Data_for_wordEmbeddings/roman.vec
reading word vectors from /content/gdrive/MyDrive/Data_for_wordEmbeddings/try_wiki.simple.vec


In [ ]:
ur_vector = ur_dictionary["acha"]
eng_vector = eng_dictionary["good"]

#print(len(ur_vector))
print(FastVector.cosine_similarity(ur_vector, eng_vector))   #since not alligned, so poor score => 0.5 shows the max similarity (Cosine function).

0.07203877131274354


In [ ]:

# from https://stackoverflow.com/questions/21030391/how-to-normalize-array-numpy
def normalized(a, axis=-1, order=2):
    """Utility function to normalize the rows of a numpy array."""
    l2 = np.atleast_1d(np.linalg.norm(a, order, axis))
    l2[l2==0] = 1
    return a / np.expand_dims(l2, axis)

def make_training_matrices(source_dictionary, target_dictionary, bilingual_dictionary):
    """
    Source and target dictionaries are the FastVector objects of
    source/target languages. bilingual_dictionary is a list of 
    translation pair tuples [(source_word, target_word), ...].
    """
    source_matrix = []
    target_matrix = []

    for (source, target) in bilingual_dictionary:
        if source in source_dictionary and target in target_dictionary:
            source_matrix.append(source_dictionary[source])
            target_matrix.append(target_dictionary[target])

    # return training matrices
    return np.array(source_matrix), np.array(target_matrix)

def learn_transformation(source_matrix, target_matrix, normalize_vectors=True):
    """
    Source and target matrices are numpy arrays, shape
    (dictionary_length, embedding_dimension). These contain paired
    word vectors from the bilingual dictionary.
    """
    # optionally normalize the training vectors
    if normalize_vectors:
        source_matrix = normalized(source_matrix)
        target_matrix = normalized(target_matrix)

    # perform the SVD
    product = np.matmul(source_matrix.transpose(), target_matrix)
    U, s, V = np.linalg.svd(product)

    # return orthogonal transformation which aligns source language to the target
    return np.matmul(U, V)

In [ ]:
ur_words = set(ur_dictionary.word2id.keys())
eng_words = set(eng_dictionary.word2id.keys())
overlap = list(ur_words & eng_words)
bilingual_dictionary = [(entry, entry) for entry in overlap]
# print(eng_dictionary.word2id.keys())
# eng_words = np.array(eng_words)
# print(eng_words(0))


eng_words_list = []
for i in eng_words:
  eng_words_list.append(i)
eng_words = np.array(eng_words_list)


In [ ]:
len(eng_words)

111048

In [ ]:
# form the training matrices
source_matrix, target_matrix = make_training_matrices(
    eng_dictionary, ur_dictionary, bilingual_dictionary)

# learn and apply the transformation
transform = learn_transformation(source_matrix, target_matrix)
eng_dictionary.apply_transform(transform)

In [ ]:
#before
eng_vector = eng_dictionary["no"]
ur_vector = ur_dictionary["nahi"]
print(FastVector.cosine_similarity(eng_vector, ur_vector))  # ~ 0.5 is the max similarity

0.35547817362397727


In [ ]:
#with urdu transform

eng_vector = eng_dictionary["no"]
ur_vector = ur_dictionary["nahi"]
print(FastVector.cosine_similarity(eng_vector, ur_vector))

0.35547817362397727


In [ ]:
eng_vector = eng_dictionary["no"]
ur_vector = ur_dictionary["han"]
print(FastVector.cosine_similarity(eng_vector, ur_vector))

0.23291628691549376


In [ ]:
len(eng_words)

file = open("/content/gdrive/MyDrive/Data_for_wordEmbeddings/aligned_vectors.vec", "w")

file.write(str(len(eng_words))+' '+ str(300) +'\n')


temp=''
for i in range(len(eng_words)):
  temp=temp + eng_words[i] + ' '
  for j in eng_dictionary[eng_words[i]]:
      # print(j ," ") 
      temp = temp + str(j) + ' '
  file.write(temp)
  file.write("\n")
  temp=''

file.close()

In [ ]:
test_dictionary = FastVector(vector_file='/content/gdrive/MyDrive/Data_for_wordEmbeddings/aligned_vectors.vec')

reading word vectors from /content/gdrive/MyDrive/Data_for_wordEmbeddings/aligned_vectors.vec


In [ ]:
test_vector = test_dictionary["no"]
ur_vector = ur_dictionary["han"]
print(FastVector.cosine_similarity(test_vector, ur_vector))

0.23291628691549376
